# Preparation

In [1]:
!pip install pygame

     |████████████████████████████████| 21.8 MB 4.4 MB/s 


In [2]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

pygame 2.1.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(640x480x32 SW)>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Thesis_Simulation/Avg base case/No connection

Mounted at /content/gdrive
/content/gdrive/MyDrive/Thesis_Simulation/Avg base case/No connection


In [4]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'../..')

from trafficSimulator import *
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

## Simulation setup

### Simulation params setup

In [5]:
all_routes = [[0, 1], [2, 3]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

vehicle_rate = 60
fast_track_factor, slow_track_factor = 1, 0.33
stop_distance = 25
fast_wait_time = 4
slow_wait_time = 4
vehicle_limit = 80

df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.5, 0.5]]*vehicle_limit))

### Topology setup

In [6]:
def parallel_line(current_coordinates, distance):
    """
    Returns the coordinate sets of lines that are parallel to the given line
    with the speicified distance
    
    Input:
        current_coordinates: two set of x and y coordinates that mark the start
                             and end of the given line
        distance: the distance between the given line and the parallel lines to
                  to be output
    """
    
    x1, y1 = current_coordinates[0]
    x2, y2 = current_coordinates[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    
    cross_x = -b/k
    hypo = math.sqrt(b**2+cross_x**2)
    
    if k>0:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))

        new_coord_2 = ((x2-distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
        
    else:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
    
        new_coord_2 = ((x2-distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))
    
    return new_coord_1, new_coord_2

In [7]:
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

bottom_left_inbound = parallel_line(current_coordinates=left_bottom_outbound, 
                                    distance=2.5)[0]
right_bottom_inbound = parallel_line(current_coordinates=bottom_right_outbound, 
                                     distance=2.5)[1]

top_left_inbound = parallel_line(current_coordinates=left_top_outbound, 
                                 distance=2.5)[0]
right_top_inbound = parallel_line(current_coordinates=top_right_outbound, 
                                  distance=2.5)[1]

### Utility params setup

In [8]:
# Smaller delta encourages more exploration
delta = 0.25

# Penalty proportion for delay caused
alpha = 0

# # Interpolation factor between money and time
# gamma_mean, gamma_var = 5, 0.5

# # Level of risk aversion
# eta_mean, eta_var = 1, 0.2

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, all_routes, vehicle_limit, df, vehicle_preferences,
                    vehicle_rate, vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global bottom_left_inbound
    global right_bottom_inbound
    global top_left_inbound
    global right_top_inbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        # A dictionary that stores the vehicles' labels as keys and their list of probabilities
        # of choosing each possible route as values
        # For the initial few rounds all vehicles choose all routes with equal weights.
        # For more complicated topologies some other algorithms will be needed as all possible
        # routes are not as explicit as in here.
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes

        # Outbound corners
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 2min 54s, sys: 4.75 s, total: 2min 59s
Wall time: 2min 57s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[2, 3]",67.75,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[0, 1]",67.4,"[-999, -999]","[0, 0]","[0, 0]"
2,2,"[2, 3]",70.85,"[0, 0]","[0, 0]","[0, 0]"
3,3,"[2, 3]",72.7,"[2, 2]","[0, 0]","[0, 0]"
4,4,"[0, 1]",70.1,"[1, 1]","[0, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[0, 1]",102.5,"[71, 71]","[0, 0]","[0, 0]"
76,76,"[2, 3]",150.95,"[74, 74]","[0, 0]","[0, 0]"
77,77,"[2, 3]",153.25,"[76, 76]","[0, 0]","[0, 0]"
78,78,"[0, 1]",103.55,"[75, 75]","[0, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[2, 3]",67.75,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[0, 1]",67.4,"[-999, -999]","[0, 0]","[0, 0]",1
2,2,"[2, 3]",70.85,"[0, 0]","[0, 0]","[0, 0]",1
3,3,"[2, 3]",72.7,"[2, 2]","[0, 0]","[0, 0]",1
4,4,"[0, 1]",70.1,"[1, 1]","[0, 0]","[0, 0]",1
5,5,"[2, 3]",74.25,"[3, 3]","[0, 0]","[0, 0]",1
6,6,"[0, 1]",71.3,"[4, 4]","[0, 0]","[0, 0]",1
7,7,"[0, 1]",72.0,"[6, 6]","[0, 0]","[0, 0]",1
8,8,"[2, 3]",75.75,"[5, 5]","[0, 0]","[0, 0]",1
9,9,"[0, 1]",72.5,"[7, 7]","[0, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.read_csv("../Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0]","[0, 0]",[],4.839465,0.845395
1,1,"[0, 0]","[0, 0]",[],4.443160,0.974247
2,2,"[0, 0]","[0, 0]",[],5.180614,1.042411
3,3,"[0, 0]","[0, 0]",[],4.615504,0.619454
4,4,"[0, 0]","[0, 0]",[],5.704076,0.777273


## Summary of the 1st round

In [14]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,1,0.0,1,67.75
1,1,0,0.0,1,67.40
2,2,1,0.0,1,70.85
3,3,1,0.0,1,72.70
4,4,0,0.0,1,70.10
5,5,1,0.0,1,74.25
6,6,0,0.0,1,71.30
7,7,0,0.0,1,72.00
8,8,1,0.0,1,75.75
9,9,0,0.0,1,72.50


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[16.651092223153956, -2.1615808438963597]","[0.9999999932429071, 6.757092967290044e-09]",[1],4.839465,0.845395
1,1,"[-3.6380134590867206, 16.651092223153956]","[1.5436660713232714e-09, 0.9999999984563339]",[0],4.443160,0.974247
2,2,"[16.651092223153956, -5.045723271067453]","[0.9999999996222603, 3.7773980850917503e-10]",[1],5.180614,1.042411
3,3,"[16.651092223153956, -0.6753333285372347]","[0.9999999701304294, 2.9869570566805395e-08]",[1],4.615504,0.619454
4,4,"[-1.6424348652141199, 16.651092223153956]","[1.1355914814553922e-08, 0.9999999886440852]",[0],5.704076,0.777273
5,5,"[16.651092223153956, -5.696175012570585]","[0.9999999998028916, 1.9710840961808552e-10]",[1],4.460208,1.078993
6,6,"[-4.297431235455103, 16.651092223153956]","[7.983105295422039e-10, 0.9999999992016894]",[0],3.981320,1.018988
7,7,"[-18.86757523322211, 16.651092223153956]","[3.753519869196378e-16, 0.9999999999999996]",[0],5.193025,1.358450
8,8,"[16.651092223153956, -3.02298720347593]","[0.999999997144673, 2.855327099965097e-09]",[1],5.120506,0.915914
9,9,"[-4.419898693807088, 16.651092223153956]","[7.062930230945256e-10, 0.999999999293707]",[0],5.128750,1.009336


In [15]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [16]:
%%time

import time
time_l = []
begin = time.time()

# Updated probabilities for the 2nd round:
prob_l = utility_df.Probabilities.values
hardmax_l = [l.index(max(l)) for l in prob_l]

prob_array = np.zeros((len(prob_l), len(all_routes)))
prob_array[np.arange(len(prob_l)), hardmax_l] = 1

vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    prob_l = utility_df.Probabilities.values
    hardmax_l = [l.index(max(l)) for l in prob_l]

    prob_array = np.zeros((len(prob_l), len(all_routes)))
    prob_array[np.arange(len(prob_l)), hardmax_l] = 1
    
    vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)
    
    time_l.append(round(time.time()-begin-sum(time_l), 2))

CPU times: user 2h 20min 32s, sys: 3min 55s, total: 2h 24min 27s
Wall time: 2h 18min 58s
